In [57]:
import polars as pl
import numpy as np
import pandas as pd

In [58]:
df=pd.read_csv("Broadband_Speed_Merged_All.csv")
df["geoid"]=df["geoid"].astype('int64')
df.head(10)

,geoid,min_d,min_u,minf_d,minf_u,maxf_d,maxf_u
0,170010001001000,2.0,0.064,2.0,3.0,1000.0,1000.0
1,170010001001001,2.0,0.064,2.0,3.0,1000.0,3.0
2,170010001001002,2.0,0.064,2.0,3.0,2.0,3.0
3,170010001001003,2.0,0.064,2.0,3.0,1000.0,3.0
4,170010001001004,2.0,0.064,2.0,3.0,2.0,3.0
5,170010001001005,2.0,0.064,2.0,3.0,2.0,3.0
6,170010001001006,2.0,0.064,2.0,3.0,2.0,3.0
7,170010001001007,2.0,0.064,2.0,3.0,1000.0,3.0
8,170010001001008,2.0,0.064,2.0,3.0,2.0,3.0
9,170010001001009,2.0,0.064,2.0,3.0,1000.0,3.0


In [59]:
df.shape,df.dtypes

((971721, 7),
 geoid       int64
 min_d     float64
 min_u     float64
 minf_d    float64
 minf_u    float64
 maxf_d    float64
 maxf_u    float64
 dtype: object)

In [60]:
index_df=pd.read_csv("data/17031-Block-Car-OD-index_166987_V2.csv")
index_df["geoid"]=index_df["geoid"].astype('int64')
index_df.head()

,id,geoid
0,1,170310101001000
1,2,170310101001001
2,3,170310101001002
3,4,170310101001003
4,5,170310101002000


In [61]:
index_df.shape,index_df.dtypes

((166986, 2),
 id       int64
 geoid    int64
 dtype: object)

In [62]:
df=df.merge(index_df, on='geoid')
df.head()

,geoid,min_d,min_u,minf_d,minf_u,maxf_d,maxf_u,id
0,170310101001000,2.0,0.384,1000.0,20.0,1000.0,20.0,1
1,170310101001001,2.0,1.300,1000.0,20.0,1000.0,20.0,2
2,170310101001002,2.0,1.300,1000.0,20.0,1000.0,20.0,3
3,170310101001003,2.0,0.512,1000.0,20.0,1000.0,20.0,4
4,170310101002000,2.0,0.384,1000.0,20.0,1000.0,20.0,5


In [63]:
df['min_a'] = df[['min_d','min_u']].min(axis=1)
df['min_f'] = df[['minf_d','minf_u']].min(axis=1)
df['max_f'] = df[['maxf_d','maxf_u']].min(axis=1)
df.drop(['min_d','min_u','minf_d','minf_u','maxf_d','maxf_u'],axis=1,inplace=True)
df.head()

,geoid,id,min_a,min_f,max_f
0,170310101001000,1,0.384,20.0,20.0
1,170310101001001,2,1.300,20.0,20.0
2,170310101001002,3,1.300,20.0,20.0
3,170310101001003,4,0.512,20.0,20.0
4,170310101002000,5,0.384,20.0,20.0


In [64]:
df.shape

(166986, 5)

In [65]:
df.to_csv("Results/Broadband_Speed_Merged_Min_Cookcounty_15mi.csv",index=False)

In [66]:
for id, df_i in  enumerate(np.array_split(df, 100)):
    df_i.to_csv('Results/Broadband_Speed_Merged_Min_Cookcounty_15mi_split/Broadband_Speed_Merged_Min_Cookcounty_15mi_{id}.csv'.format(id=id),index=False)

In [67]:
# zoom 1:1 video bandwidth requirement
#zoom_up=[0,0.6,1.2,3.8,9999.0]
zoom_down=[0,0.6,1.2,3.0,9999.0]
zoom_weight=[0,0.33,0.66,1]

In [ ]:
#assume that zoom download and updload are the same requirements
df["min_d_w"] = pd.cut(df["min_d"], right=False,bins=zoom_down, labels=zoom_weight)
df["minf_d_w"] = pd.cut(df["minf_d"],right=False, bins=zoom_down, labels=zoom_weight)
df["maxf_d_w"] = pd.cut(df["maxf_d"], right=False,bins=zoom_down, labels=zoom_weight)

df["min_u_w"] = pd.cut(df["min_u"], right=False,bins=zoom_down, labels=zoom_weight)
df["minf_u_w"] = pd.cut(df["minf_u"], right=False,bins=zoom_down, labels=zoom_weight)
df["maxf_u_w"] = pd.cut(df["maxf_u"], right=False,bins=zoom_down, labels=zoom_weight)
df.head()

## Crossjoin

In [46]:
import glob
csv_list=glob.glob("Results/Broadband_Speed_Merged_Min_Cookcounty_15mi_split/Broadband_Speed_Merged_Min_Cookcounty_15mi_*.csv")

In [47]:
df=pl.read_csv("IL_Broadband_Speed_Merged_Min.csv").lazy()
df.head(5)

In [59]:
for csv_i in csv_list:
    df_i=pl.read_csv(csv_i).lazy()
    temp_df=df_i.join(df,on="geoid",how="cross")
    out_df=temp_df.select(
        [
            pl.col("geoid").alias("orig"),
            pl.col("geoid_right").alias("dest"),
            pl.min(["min_a","min_a_right"]).alias("min_a"),
            pl.min(["min_f","min_f_right"]).alias("min_f"),
            pl.min(["max_f","max_f_right"]).alias("max_f"),
        ]
    ).collect().to_pandas()
    #print(out_df.head(5))

    out_df["min_a_w"] = pd.cut(out_df["min_a"], right=False,bins=zoom_down, labels=zoom_weight)
    out_df["min_f_w"] = pd.cut(out_df["min_f"], right=False,bins=zoom_down, labels=zoom_weight)
    out_df["max_f_w"] = pd.cut(out_df["max_f"], right=False,bins=zoom_down, labels=zoom_weight)
    out_df.drop(["min_a","min_f","max_f"],axis=1,inplace=True)
    out_df.to_csv("/Volumes/Mac Data/joined_data/"+csv_i,index=False)
    print("joined_data/"+csv_i)
    #print(out_df.head(5))
    #break;

joined_data/data/IL_Broadband_Speed_Merged_Min_73.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_67.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_98.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_99.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_66.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_72.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_64.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_70.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_58.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_59.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_71.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_65.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_8.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_49.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_61.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_75.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_74.csv
joined_data/data/IL_Broadband_Speed_Merged_Min_60.csv
joined_data/data/IL_Broadband

## Computed using polars

In [70]:
import glob
csv_list=glob.glob("Results/Broadband_Speed_Merged_Min_Cookcounty_15mi_split/Broadband_Speed_Merged_Min_Cookcounty_15mi_*.csv")
df=pl.read_csv("Results/Broadband_Speed_Merged_Min_Cookcounty_15mi.csv").lazy()
df.head(5)

In [73]:
i=0
for csv_i in csv_list:
    df_i=pl.read_csv(csv_i).lazy()
    temp_df=df_i.join(df,on="geoid",how="cross")
    out_df=temp_df.select(
        [
            pl.col("geoid").alias("orig"),
            pl.col("geoid_right").alias("dest"),
            pl.min(["min_a","min_a_right"]).alias("min_a_w"),
            pl.min(["min_f","min_f_right"]).alias("min_f_w"),
            pl.min(["max_f","max_f_right"]).alias("max_f_w"),
        ]
    ).collect()
    #print(out_df.head(5))
    
    out_df2=out_df.with_column(
        pl.when(pl.col(["min_a_w","min_f_w","max_f_w"]) <= 0.6)
        .then(0.0)
        .when(pl.col(["min_a_w","min_f_w","max_f_w"]) <= 1.2)
        .then(0.33)
        .when(pl.col(["min_a_w","min_f_w","max_f_w"]) <= 3.0)
        .then(0.66)
        .otherwise(1)
    )

    out_df2=out_df2.drop(["min_a_w","min_f_w","max_f_w"])
    out_df2.columns=["orig","dest","min_a_w","min_f_w","max_f_w"]
    out_df2.write_csv("/Volumes/easystore/3.Broadband_Speed_E2SVCA_15MI_BLOCKS/"+str(i)+".csv")
    print("/Volumes/easystore/3.Broadband_Speed_E2SVCA_15MI_BLOCKS/"+str(i)+".csv")
    i++;
    #print(out_df2.head(5))
    #break;

/Volumes/easystore/3.Broadband_Speed_E2SVCA_15MI_BLOCKS/0.csv


## Merged data results

In [4]:
df = pl.scan_csv("/Volumes/Mac Data/joined_data/data/IL_Broadband_Speed_Merged_Min_*.csv").collect()
df.write_csv("/Volumes/Mac Data/joined_data/IL_Broadband_Speed_Merged_Min_all.csv")
#print(df)

Error: Canceled future for execute_request message before replies were done